In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

2023-01-02 12:28:49.495721: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-02 12:28:50.468602: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-02 12:28:50.468661: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-02 12:28:54.360463: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [9]:
df = pd.read_csv("data/diabetes.csv")
df.pop("Outcome")

columns = df.columns.values
rand = random.randint(0, len(columns))

features = df.values[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'Pregnancies': 0.0,
 'Glucose': 137.0,
 'BloodPressure': 40.0,
 'SkinThickness': 35.0,
 'Insulin': 168.0,
 'BMI': 43.1,
 'DiabetesPedigreeFunction': 2.288,
 'Age': 33.0}

In [10]:
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

In [11]:
def prepare_json(inputs: dict):
    feature_spec = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, int):
            feature_spec[keys] = int_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [21]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)
    
    endpoint = "https://diabetes-pred.herokuapp.com/v1/models/diabetes-disease-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    
    if prediction < 0.6:
        return "Normal"
    else:
        return "Diabetes"

In [20]:
make_predictions(inputs)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)